**News Classification by LSTM**

In this notebook, we will try to classify news merely from the language associated with it. We just use its headline and short description to classify the news's category. One thing we intentionally avoid is the author's name due to tendency of certain author to write articles on particular topics.

In general this notebook is comprised of some sections which are:
1. Preparing data
2. Building the model
3. Training the model
4. User input

We use some components here to name a few:
* Torchtext library
* Pre-trained word embedding
* LSTM network architecture
* Bidirectional LSTM
* Multi-layered LSTM
* Regularization
* Adam optimizer
* Cross-entropy loss function for classification problem

**Preparing Data**

We use Torchtext library to pre-process our data. Torchtext simplifies text data pre-processing that includes reading data, tokenizing, converting into tensors, and building vocabulary to be easier.

In [40]:
import torch
from torchtext.legacy import data

So first we specify what our data comprises of. We decide that our data comprises of TEXT which are the news' headlines and short descriptions, as well as LABEL which is the news' category. Here we tokenize the text using [spacy](https://spacy.io/?source=post_page---------------------------) tokenizer and to make all the words use lower case. While we keep the entire LABEL as it is.

In [41]:
TEXT = data.Field(tokenize = 'spacy', lower = True)
LABEL = data.LabelField()

Let's use TabularDataset for json type file here. We extract the entirety of our data into something like dictionary with three keys, 'headline', 'desc', and 'category' that corresponds to each news' headline, short description, and category.

In [42]:
news = data.TabularDataset(
    path='input/News_Category_Dataset_v2.json', format='json',
    fields={'headline': ('headline', TEXT),
            'short_description' : ('desc', TEXT),
             'category': ('category', LABEL)})

Further split our dataset into training set trn, validation set vld, and test set tst using seed for reproducible result.

In [43]:
import random
SEED = 1234
from tqdm import tqdm_notebook, tqdm

trn, vld, tst = news.split(split_ratio=[0.7, 0.2, 0.1], random_state = random.seed(SEED))

We will check an example of our data. It should comprises parsed headline, description, and the associated category.

In [44]:
vars(trn[0])

{'headline': ["'", 'train', 'hard', ',', 'land', 'soft', "'"],
 'desc': ['runners',
  'will',
  'appreciate',
  'that',
  'the',
  'sproing',
  'trainer',
  'was',
  'designed',
  'with',
  'them',
  'in',
  'mind',
  'as',
  'a',
  'way',
  'to',
  'build',
  'endurance',
  'and',
  'strength',
  'without',
  'the',
  'pain',
  'that',
  'can',
  'come',
  'with',
  'pounding',
  'the',
  'pavement',
  '.'],
 'category': 'HEALTHY LIVING'}

We build our vocabulary from our datasets and convert it into vectors from glove. From there we check how many vocabularies we have from our text and how many categories we have.

In [45]:
TEXT.build_vocab(trn, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(trn)

In [46]:
print(len(TEXT.vocab))
print(len(LABEL.vocab))

78590
41


In [47]:
print(LABEL.vocab.stoi)

defaultdict(None, {'POLITICS': 0, 'WELLNESS': 1, 'ENTERTAINMENT': 2, 'TRAVEL': 3, 'STYLE & BEAUTY': 4, 'PARENTING': 5, 'HEALTHY LIVING': 6, 'QUEER VOICES': 7, 'FOOD & DRINK': 8, 'BUSINESS': 9, 'COMEDY': 10, 'SPORTS': 11, 'BLACK VOICES': 12, 'HOME & LIVING': 13, 'PARENTS': 14, 'THE WORLDPOST': 15, 'WEDDINGS': 16, 'DIVORCE': 17, 'WOMEN': 18, 'IMPACT': 19, 'CRIME': 20, 'MEDIA': 21, 'WEIRD NEWS': 22, 'GREEN': 23, 'WORLDPOST': 24, 'RELIGION': 25, 'STYLE': 26, 'SCIENCE': 27, 'TASTE': 28, 'WORLD NEWS': 29, 'TECH': 30, 'MONEY': 31, 'ARTS': 32, 'FIFTY': 33, 'GOOD NEWS': 34, 'ARTS & CULTURE': 35, 'ENVIRONMENT': 36, 'COLLEGE': 37, 'LATINO VOICES': 38, 'CULTURE & ARTS': 39, 'EDUCATION': 40})


Here, let's wrap out data to get the relevant iterator for our training, validation, as well as test sets.

In [48]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (trn, vld, tst), 
    batch_size = BATCH_SIZE, 
    device = device,
    sort_key= lambda x: len(x.headline), 
    sort_within_batch= False
    )

cuda


**Building the Model**

In this section, we define our model. Since we are trying to classify the news based on its headline and short description that are in the form of sentences or paragraphs, we are going to use sequential model that is LSTM (Long Short Term Memory). More specifically, we use bidirectional and two-layered LSTM layer hopefully to get better accuracy for our prediction. We also implement regularization by using dropout during our forward pass. In this model, we specifically split the processing for the headline and short description and concatenate them before final processing to get the prediction of our news' category. The detail can be seen in the diagram below:

![](https://i.imgur.com/6nXjqx8.png)

In [49]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        
        super().__init__()
                
        self.embedding = nn.Embedding(vocab_size, embedding_dim).to(device)
        
        self.lstm_head = nn.LSTM(embedding_dim, hidden_dim, num_layers = n_layers, bidirectional = bidirectional, dropout = dropout).to(device)
        
        self.lstm_desc = nn.LSTM(embedding_dim, hidden_dim, num_layers = n_layers, bidirectional = bidirectional, dropout = dropout).to(device)
        
        self.fc_head = nn.Linear(hidden_dim * 2, 100).to(device)
        
        self.fc_desc = nn.Linear(hidden_dim * 2, 100).to(device)

        self.fc_total = nn.Linear(200, output_dim).to(device)
        
        self.dropout = nn.Dropout(dropout).to(device)
                
    def forward(self, headline, description):
                        
        embedded_head = self.dropout(self.embedding(headline))
        
        embedded_desc = self.dropout(self.embedding(description))
                                    
        output_head, (hidden_head, cell_head) = self.lstm_head(embedded_head)
        
        output_desc, (hidden_desc, cell_desc) = self.lstm_desc(embedded_desc)
        
        hidden_head = self.dropout(torch.cat((hidden_head[-2, :, :], hidden_head[-1, :, :]), dim = 1))
        
        hidden_desc = self.dropout(torch.cat((hidden_desc[-2, :, :], hidden_desc[-1, :, :]), dim = 1))
        
        full_head = self.fc_head(hidden_head)
        
        full_desc = self.fc_desc(hidden_desc)
        
        hidden_total = torch.cat((full_head, full_desc), 1)
        
        return self.fc_total(hidden_total)

Now we create our model and check how many parameters we are training.

In [50]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = len(LABEL.vocab)
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.2

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT)

In [51]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 12,590,129 trainable parameters


Next, replace the initial weights of the embedding layers with the pre-trained embeddings.

In [52]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([78590, 100])


In [53]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 1.7231,  2.0242, -0.2619,  ..., -0.7982,  0.0636, -0.0275],
        [-2.1380,  0.2913, -1.1321,  ...,  0.9435, -0.8038,  1.0310],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.6361, -1.4962,  0.6593,  ...,  1.0944,  0.6592,  0.1426],
        [-0.6850, -0.3640,  0.5942,  ..., -1.0381,  0.7413, -0.8590],
        [ 0.3260,  0.9341,  0.7865,  ...,  2.3977, -1.2733,  0.6887]],
       device='cuda:0')

**Training the Model**

We choose Adam algorithm as our optimizer, as well as cross entropy loss for our loss function since we are doing classification problem with multiple categories. We also define the function to calculate accuracy of our prediction.

In [54]:
def categorical_accuracy(preds, y):
    max_preds = preds.argmax(dim = 1, keepdim = True).to(device)
    correct = max_preds.squeeze(1).eq(y).to(device)
    return correct.sum() / torch.FloatTensor([y.shape[0]]).to(device)

Here we define the training and evaluate part of our model.

In [55]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()

    # Get the progress bar for later modification
    # progress_bar = tqdm_notebook(iterator, ascii=True)
    for idx, batch in enumerate(iterator):
        
        optimizer.zero_grad()
                        
        predictions = model(batch.headline, batch.desc)
#         print(predictions.shape)
        predictions = predictions.squeeze(1)
#         print(batch.category)
        
        loss = criterion(predictions, batch.category)
        
        acc = categorical_accuracy(predictions, batch.category)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()

        # progress_bar.set_description_str(
        #     "Batch: %d, Loss: %.4f" % (int(idx + 1), loss.item()))
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [56]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:
            
            predictions = model(batch.headline, batch.desc).squeeze(1)
            
            loss = criterion(predictions, batch.category)
            
            acc = categorical_accuracy(predictions, batch.category)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [57]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [58]:
import numpy as np

cls_num_list = []
cat_map = dict()
for r in news:
    cat = r.category
    if cat not in cat_map:
        cat_map[cat] = 0
    else:
        cat_map[cat] += 1
for v in cat_map.values():
    cls_num_list.append(v)
cls_num_list = sorted(cls_num_list, reverse=True)

Now we are ready to train our model. We will train it for five epochs.

In [61]:
# pure default (no scheduler/comment out) gives the overfitted one
from losses.LDAMLoss import FocalLoss
import torch.optim as optim
lr = 0.0002
betas = [0.9, 0.99999]
weight_decay = 0.0000
# lr = 0.001
# momentum = 0.9
optimizer = optim.Adam(model.parameters(), lr=lr, betas=betas, weight_decay=weight_decay)  #, betas=betas)
# optimizer = torch.optim.SGD(model.parameters(), lr,
#                             momentum=momentum,
#                             weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                       patience=1, threshold=.5,
                                                       cooldown=0, factor=0.1)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5, last_epoch=-1)
# lr_idx = [0.0005, 0.0005, 0.0001, 0.0001]
betas = [0, 0.9999]
idx = 0
effective_num = 1.0 - np.power(betas[idx], cls_num_list)
per_cls_weights = (1.0 - betas[idx]) / np.array(effective_num)
per_cls_weights = per_cls_weights / np.sum(per_cls_weights) * len(cls_num_list)
per_cls_weights = torch.FloatTensor(per_cls_weights).to(device)
# per_cls_weights = per_cls_weights.detach().numpy()
criterion = FocalLoss(weight=per_cls_weights, gamma=0.5, reduction='mean')

In [ ]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    model = model.to(device)
    criterion = criterion.to(device)

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    scheduler.step(train_loss)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'news_classification_model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    llrr = optimizer.param_groups[0]['lr']
    print(f'\t LR: {llrr:.10f}')
    # optimizer.param_groups[0]['lr'] = lr_idx[epoch]

Epoch: 01 | Epoch Time: 1m 3s
	Train Loss: 1.071 | Train Acc: 63.13%
	 Val. Loss: 1.135 |  Val. Acc: 62.02%
	 LR: 0.0002000000
Epoch: 02 | Epoch Time: 1m 3s
	Train Loss: 0.970 | Train Acc: 65.67%
	 Val. Loss: 1.088 |  Val. Acc: 63.32%
	 LR: 0.0002000000
Epoch: 03 | Epoch Time: 1m 5s
	Train Loss: 0.885 | Train Acc: 67.68%
	 Val. Loss: 1.048 |  Val. Acc: 64.23%
	 LR: 0.0000200000
Epoch: 04 | Epoch Time: 1m 5s
	Train Loss: 0.771 | Train Acc: 70.69%
	 Val. Loss: 1.022 |  Val. Acc: 65.10%
	 LR: 0.0000200000


And we test it with our best model.

In [ ]:
test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

**User Input**

In this section, we let ourself to put our own input and let the model predict the news' categories beyond the dataset. For consistencies, we will use news from Huffington Post and try to get its category predicted. Make sure that the first input is the headline and the second input is the short description of the article.

News can be obtained from [here](https://www.huffpost.com/).

In [ ]:
import spacy
nlp = spacy.load('en')

def predict_category(model, head, desc):
    model.eval()
    head = head.lower()
    desc = desc.lower()
    tokenized_head = [tok.text for tok in nlp.tokenizer(head)]
    tokenized_desc = [tok.text for tok in nlp.tokenizer(desc)]
    indexed_head = [TEXT.vocab.stoi[t] for t in tokenized_head]
    indexed_desc = [TEXT.vocab.stoi[t] for t in tokenized_desc]
    tensor_head = torch.LongTensor(indexed_head).to(device)
    tensor_desc = torch.LongTensor(indexed_desc).to(device)
    tensor_head = tensor_head.unsqueeze(1)
    tensor_desc = tensor_desc.unsqueeze(1)
    prediction = model(tensor_head, tensor_desc)
    max_pred = prediction.argmax(dim=1)
    return max_pred.item()

News headline: Trump’s Art Of Distraction

News short description: The conversation surrounding Trump’s latest racist rants has provoked us to revisit author Toni Morrison’s 1975 keynote address at Portland State University on the true purpose of racism.

Correct category: Politics

In [ ]:
pred = predict_category(model, "Trump’s Art Of Distraction", "The conversation surrounding Trump’s latest racist rants has provoked us to revisit author Toni Morrison’s 1975 keynote address at Portland State University on the true purpose of racism..")
print(f'Predicted category is: {pred} = {LABEL.vocab.itos[pred]}')

News headline: Indiana Cop Apologizes After Accusing McDonald’s Worker Of Eating His Sandwich

News short description: The Marion County sheriff’s deputy forgot he had taken a bite out of his McChicken earlier that day, authorities said.

Correct category: U.S. News

In [ ]:
pred = predict_category(model, "Indiana Cop Apologizes After Accusing McDonald’s Worker Of Eating His Sandwich", "The Marion County sheriff’s deputy forgot he had taken a bite out of his McChicken earlier that day, authorities said.")
print(f'Predicted category is: {pred} = {LABEL.vocab.itos[pred]}')

News headline: Kyle ‘Bugha’ Giersdorf, 16, Wins Fortnite World Cup And Takes Home $ 3 Million Prize

News short description: Fortnite has nearly 250 million registered players and raked in an estimated $2.4 billion last year.

Correct category: Sports

In [ ]:
pred = predict_category(model, "Kyle ‘Bugha’ Giersdorf, 16, Wins Fortnite World Cup And Takes Home $ 3 Million Prize", "Fortnite has nearly 250 million registered players and raked in an estimated $2.4 billion last year.")
print(f'Predicted category is: {pred} = {LABEL.vocab.itos[pred]}')

**References**

This notebook was created thanks to the two references below.
* http://mlexplained.com/2018/02/08/a-comprehensive-tutorial-to-torchtext/
* https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/2%20-%20Upgraded%20Sentiment%20Analysis.ipynb